## USE CASE GENERATION USING LLM

### Approach


                    text ip          use case          plantUML script
Problem statement -----------> LLM ------------> LLM ---------------------> plantUML -----> Usecase Diagram
                     fewshot           desc
                     cot

In [1]:
import os
from langchain_openai import ChatOpenAI  # OpenAI integration
from langchain_core.prompts import PromptTemplate  # Core prompt handling 
from langchain.chains import LLMChain

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the API key in LangChain/OpenAI
os.environ["OPENAI_API_KEY"] = openai_api_key

### PROBLEM STATEMENT

In [3]:
problem_statement = """
An Airline Ticketing System allows the Clerk to order tickets and to look for a suitable flight. In order to
look up a ticket, the Clerk needs to enter the name of the traveler, the chosen flight, and finally booking
the airline tickets. When looking up a flight, the system provides a list of available flights between two airports
at a given date. There is also an option of multicity, where the customer chooses to fly through multiple
destinations with halting. The system must allow the ability to look up a flight without having to order a
ticket. When the order information to buy a ticket is entered, this information is automatically used to
look up the flight.
A third service (the help service) is available for the clerk while ordering a ticket to provide a help page
specific to ordering tickets. The help service can be invoked independently by the Clerk, which will provide
the Clerk with a “main” help page.
"""

### FEW SHOT EXAMPLE

In [4]:
use_case_few_shot_prompt = PromptTemplate(
    input_variables=["problem_statement"],
    template="""
Assume you are a software engineer developing structured use case models for a System.
Follow these structured rules while generating use cases:

1. Use Case Naming:  
   - Use an action verb followed by a qualified object (e.g., "Request Ticket", "Process Payment").  
   - Avoid vague verbs (e.g., "do", "process") and database-related terms (CRUD operations).  

2. Actors Identification:  
   - Primary Actor: The entity that initiates the interaction.  
   - Secondary Actors: Supporting entities involved in the process.  

3. Use Case Structure:  
   - Preconditions: The necessary conditions before execution.  
   - Main Flow: Clearly sequential interactions, using **subject-verb-object** structure.  
   - Alternative Flow: Document possible variations and edge cases.  
   - Exception Handling: Define failure scenarios and recovery strategies.  
   - Postconditions: Expected outcome after successful execution.  

4. Language & Formatting: 
   - Use active voice and declarative sentences.  
   - Avoid negations (e.g., "not", "never") unless required.  
   - Use present tense and maintain consistency.  
   - No GUI elements should be mentioned.  
   - Avoid actor-to-actor interactions (all interactions must go through the system).  

## FEW-SHOT EXAMPLES ##
### Example Use Case ###
Use Case Name: Request Airline Ticket
Primary Actor: Clerk  
Secondary Actors: Payment Gateway, Airline Reservation System  
Preconditions:  
1. The Clerk must be authenticated in the system.  
2. The airline must have available flights for booking.  

Main Flow:
1. Clerk selects a flight from the available list.  
2. System retrieves ticket details and prompts for passenger information.  
3. Clerk enters passenger details and confirms ticket selection.  
4. System validates the entered details.  
5. Clerk submits payment request.  
6. System forwards payment request to the Payment Gateway.  
7. Payment Gateway authorizes the transaction and confirms the payment.  
8. System generates the ticket and assigns a Passenger Name Record (PNR).  
9. System updates the airline reservation system with booking details.  
10. System notifies the Clerk of successful ticket booking.  

Alternative Flow:  
- If the payment fails, the system prompts the Clerk to retry or select another payment method.  
- If no seats are available, the system suggests alternative flights.  

Postconditions:  
1. The airline ticket is successfully booked.  
2. The passenger receives the ticket confirmation with PNR.  

---

Now generate a structured use case based on the following problem statement:  
{problem_statement}
"""
)


In [5]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
use_case_chain = LLMChain(llm=llm, prompt=use_case_few_shot_prompt)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15956\1958579587.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  use_case_chain = LLMChain(llm=llm, prompt=use_case_few_shot_prompt)


In [6]:
use_case_few_shot_output = use_case_chain.run(problem_statement=problem_statement)
print("\nGenerated Use Cases:\n")
print(use_case_few_shot_output)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15956\229859339.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  use_case_few_shot_output = use_case_chain.run(problem_statement=problem_statement)



Generated Use Cases:

### Example Use Case ###
Use Case Name: Order Airline Tickets  
Primary Actor: Clerk  
Secondary Actors: Airline Reservation System, Help Service  

Preconditions:  
1. The Clerk must be authenticated in the system.  
2. The airline must have available flights for booking.  

Main Flow:  
1. Clerk enters the name of the traveler, chosen flight, and booking details.  
2. System validates the entered details and retrieves available flights.  
3. Clerk selects a suitable flight from the list.  
4. System prompts for confirmation on flight selection.  
5. Clerk confirms the flight selection and proceeds to order tickets.  
6. System generates the ticket and assigns a Passenger Name Record (PNR).  
7. System updates the airline reservation system with booking details.  
8. System notifies the Clerk of successful ticket booking.  

Alternative Flow:  
- If the Clerk chooses the multicity option, the system provides a list of flights with halting at multiple destination

### CHAIN OF THOUGHT EXAMPLE

In [7]:
use_case_cot_prompt = PromptTemplate(
    input_variables=["problem_statement"],
    template="""
Assume you are a software engineer developing an Airline Ticketing System.
To ensure correctness, follow these steps:

1. Identify all the goals of the system.  
   Think carefully about what the system is trying to achieve and list all the goals.
   
2. For each goal, determine the primary actor (who initiates the action).  
   Think about who will trigger the action for each goal. It could be a customer, an admin, or a system.

3. Identify any secondary actors involved.  
   These are entities that play a supporting role, such as systems or external services.

4. Describe the interaction steps between actors and the system.  
   Detail the sequence of actions that take place to fulfill the goal.

5. Mention exceptions and alternative flows.  
   Consider possible exceptions or edge cases where the flow might differ from the norm.

### Example Use Case ###
Use Case: Order Ticket
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.

Steps:
1. Clerk enters passenger details.
2. Clerk selects the desired flight.
3. Airline System checks availability.
4. Clerk proceeds to booking.
5. System confirms booking and moves to payment.
6. Clerk enters payment details.
7. Payment Gateway processes payment.
8. Airline System confirms payment.
9. System generates PNR.
10. Confirmation message is sent to the Clerk or Customer.

Exceptions:
1. If payment fails, prompt the Clerk to retry.
2. If the flight is full, suggest alternative flights.
3. If the PNR cannot be generated, notify the Clerk of the issue.

Now, apply the same reasoning step-by-step to generate a use cases for:
{problem_statement}
"""
)

In [8]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
use_case_chain = LLMChain(llm=llm, prompt=use_case_cot_prompt)

In [9]:
use_case_cot_output = use_case_chain.run(problem_statement=problem_statement)
print("\nGenerated Use Cases:\n")
print(use_case_cot_output)


Generated Use Cases:

Use Case: Look Up Flight
Primary Actor: Clerk
Secondary Actors: Airline System
Preconditions: The Clerk must be logged into the system.

Steps:
1. Clerk enters departure and arrival airports and date.
2. Clerk selects the type of flight (single or multicity).
3. System provides a list of available flights.
4. Clerk reviews flight options.

Exceptions:
1. If no flights are available, notify the Clerk and suggest alternative dates or airports.

Use Case: Order Ticket with Flight Lookup
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.

Steps:
1. Clerk enters passenger details.
2. Clerk uses the 'Look Up Flight' use case to find a suitable flight.
3. Clerk selects the desired flight.
4. Airline System checks availability.
5. Clerk proceeds to booking.
6. System confirms booking and moves to payment.
7. Clerk enters payment details.
8. Payment Gateway processes payment.
9. Airline System co

In [10]:
print(use_case_few_shot_output)

### Example Use Case ###
Use Case Name: Order Airline Tickets  
Primary Actor: Clerk  
Secondary Actors: Airline Reservation System, Help Service  

Preconditions:  
1. The Clerk must be authenticated in the system.  
2. The airline must have available flights for booking.  

Main Flow:  
1. Clerk enters the name of the traveler, chosen flight, and booking details.  
2. System validates the entered details and retrieves available flights.  
3. Clerk selects a suitable flight from the list.  
4. System prompts for confirmation on flight selection.  
5. Clerk confirms the flight selection and proceeds to order tickets.  
6. System generates the ticket and assigns a Passenger Name Record (PNR).  
7. System updates the airline reservation system with booking details.  
8. System notifies the Clerk of successful ticket booking.  

Alternative Flow:  
- If the Clerk chooses the multicity option, the system provides a list of flights with halting at multiple destinations.  
- If the Clerk nee

### Generate PlantUML Script Using LLM

In [11]:

plantuml_prompt = PromptTemplate(
    input_variables=["use_case_cot_output"],
    template="""
You are an expert in system modeling. Convert the following textual use case into a valid **PlantUML script**.
Ensure that:
1. **Actors are properly defined** (Primary and Secondary).
2️. **Use case relationships (extends, includes) are accurately represented**.
3️. **Flows and dependencies between actors and use cases are logically structured**.

### Example Use Case ###
### Use Case Name: Order Airline Ticket ###
Primary Actor: Clerk  
Secondary Actors: Airline Reservation System, Payment Gateway  
Preconditions:  
1. The Clerk must be authenticated in the system.  
2. The airline must have available flights for booking.  

### Example PlantUML Output ###
@startuml
actor Clerk
actor "Payment Gateway" as PG
actor "Airline Reservation System" as ARS

Clerk -> (Order Airline Ticket)
(Order Airline Ticket) --> PG : Process Payment
(Order Airline Ticket) --> ARS : Confirm Booking

@enduml

---
Now generate a **PlantUML script** for the following textual use case:

{use_case_cot_output}
"""
)


In [12]:
import os
print("Files in current directory:", os.listdir())

Files in current directory: ['.env', 'app.py', 'app2.py', 'main.ipynb', 'plantuml-1.2025.0.jar', 'use_case_diagram.txt']


In [13]:
import subprocess
import os

def generate_uml_diagram(plantuml_script, output_file="use_case_diagram.png", jar_path="plantuml-1.2025.0.jar"):
    """Generates a UML diagram from a PlantUML script."""
    
    uml_file = "use_case_diagram.txt"
    log_file = "plantuml_log.txt"

    # Save PlantUML script to a file
    with open(uml_file, "w") as file:
        file.write(plantuml_script)
    
    # Run PlantUML and capture output/errors
    try:
        print("[*] Generating UML Diagram...")
        result = subprocess.run(
            ["java", "-jar", jar_path, uml_file], capture_output=True, text=True
        )

        # Save logs
        with open(log_file, "w") as log:
            log.write("PlantUML Output:\n" + result.stdout + "\nPlantUML Errors:\n" + result.stderr)

        print("[✔] PlantUML Output:", result.stdout)
        if result.stderr:
            print("[!] PlantUML Errors:", result.stderr)

        # Check if the output file is created
        generated_file = uml_file.replace(".txt", ".png")
        if os.path.exists(generated_file):
            os.rename(generated_file, output_file)
            print(f"[✔] UML Diagram generated: {output_file}")
        else:
            raise FileNotFoundError("PlantUML did not generate the expected output file.")

    except Exception as e:
        print(f"[✘] Error generating UML diagram: {str(e)}")
    
    # Cleanup intermediate files
    os.remove(uml_file)

# Example Usage
plantuml_script = """
@startuml
actor Clerk
actor "Payment Gateway" as PG
actor "Airline Reservation System" as ARS
participant "System" as SYS

== Main Flow ==
Clerk -> SYS : Select flight
SYS -> Clerk : Retrieve ticket details, prompt for passenger info
Clerk -> SYS : Enter passenger details, confirm selection
SYS -> SYS : Validate entered details
Clerk -> SYS : Submit payment request
SYS -> PG : Forward payment request
PG -> SYS : Authorize transaction, confirm payment
SYS -> SYS : Generate ticket, assign PNR
SYS -> ARS : Update reservation system
SYS -> Clerk : Notify of successful booking

@enduml
"""

generate_uml_diagram(plantuml_script, output_file="airline_ticket_use_case.png")


[*] Generating UML Diagram...
[✔] PlantUML Output: 
[✔] UML Diagram generated: airline_ticket_use_case.png


In [14]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from IPython.display import Image

# Step 1: Generate textual use case
def generate_textual_use_case(problem_statement):
    try:
        response = use_case_few_shot_prompt.invoke({"problem_statement": problem_statement})
        return str(response)  # Ensure string format
    except Exception as e:
        print(f"[✘] Error in generating textual use case: {str(e)}")
        return ""

# Step 2: Convert textual use case to PlantUML script
def generate_plantuml_script(use_case_cot_output):
    try:
        response = plantuml_prompt.invoke({"use_case_cot_output": use_case_cot_output})
        return str(response.text)  # Convert StringPromptValue to string
    except Exception as e:
        print(f"[✘] Error in generating PlantUML script: {str(e)}")
        return ""

# Step 3: Render Use Case Diagram
def render_use_case_diagram(plantuml_script):
    try:
        return generate_uml_diagram(str(plantuml_script))  # Ensure string format
    except Exception as e:
        print(f"[✘] Error in rendering UML diagram: {str(e)}")
        return ""

# Define LangChain Pipeline
pipeline = (
    RunnableParallel({
        "use_case_cot_output": RunnableLambda(generate_textual_use_case)
    })
    .assign(plantuml_script=lambda x: generate_plantuml_script(x["use_case_cot_output"]))
    .assign(diagram=lambda x: render_use_case_diagram(x["plantuml_script"]))
)

# Run the Full Pipeline
problem_statement = "An online learning platform allows students to enroll in courses, attend live lectures, and take quizzes. Instructors create courses and manage content."
results = pipeline.invoke(problem_statement)

# Display the generated diagram
if results["diagram"]:
    Image(filename=results["diagram"])
else:
    print("[✘] Failed to generate the UML diagram.")


[*] Generating UML Diagram...
[✔] PlantUML Output: 
[✔] UML Diagram generated: use_case_diagram.png
[✘] Failed to generate the UML diagram.
